In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
sys.path.append('../saildrone/subroutines/')
import warnings
warnings.simplefilter('ignore') # filter some warning messages
from glob import glob


# read in collocated files

In [2]:
data_dir = 'F:/data/cruise_data/saildrone/sss_collocations/'
data_dir_out = 'F:/data/cruise_data/saildrone/sss_collocations_norepeat/'
filenames = [x for x in glob(data_dir+'*.nc')]
filenames

['F:/data/cruise_data/saildrone/sss_collocations\\PMEL_Arctic_2015_sd126-ALL-1_min-v1jplv4.2_filesave3.nc',
 'F:/data/cruise_data/saildrone/sss_collocations\\PMEL_Arctic_2015_sd126-ALL-1_min-v1rssv4_filesave3.nc',
 'F:/data/cruise_data/saildrone/sss_collocations\\PMEL_Arctic_2015_sd128-ALL-1_min-v1jplv4.2_filesave3.nc',
 'F:/data/cruise_data/saildrone/sss_collocations\\PMEL_Arctic_2015_sd128-ALL-1_min-v1rssv4_filesave3.nc',
 'F:/data/cruise_data/saildrone/sss_collocations\\PMEL_Arctic_2016_sd126-ALL-1_min-v1jplv4.2_filesave3.nc',
 'F:/data/cruise_data/saildrone/sss_collocations\\PMEL_Arctic_2016_sd126-ALL-1_min-v1rssv4_filesave3.nc',
 'F:/data/cruise_data/saildrone/sss_collocations\\PMEL_Arctic_2016_sd128-ALL-1_min-v1jplv4.2_filesave3.nc',
 'F:/data/cruise_data/saildrone/sss_collocations\\PMEL_Arctic_2016_sd128-ALL-1_min-v1rssv4_filesave3.nc',
 'F:/data/cruise_data/saildrone/sss_collocations\\saildrone-gen_4-baja_2018-sd1002-20180411T180000-20180611T055959-1_minutes-v1jplv4.2_filesave3

In [3]:
for iname,name in enumerate(filenames):
    i = name.find('\\')
    fout = data_dir_out + name[i+1:-3]+'norep.nc' 
    ds = xr.open_dataset(name)
    ds.close()
    ds_tem2 = ds.where((ds.smap_SSS<50) & (ds.smap_SSS>1),drop=True)
    isv=0
    while len(ds_tem2.time)>1:
        i=0
        cond = ((ds_tem2.smap_name==ds_tem2.smap_name[i]) 
                & (ds_tem2.smap_ydim==ds_tem2.smap_ydim[i]) 
                & (ds_tem2.smap_xdim==ds_tem2.smap_xdim[i]))
        subset = ds_tem2.where(cond,drop=True)  #repeat obs
        ds_mn = subset.mean(keep_attrs=True,skipna=True)
        ds_mn['time'] = subset.time.mean()
        ds_mn = ds_mn.assign_coords({'ob':isv})
        if isv==0:
            ds_mn2 = ds_mn
        else:
            ds_mn2 = xr.concat([ds_mn2,ds_mn],dim='ob')
        isv = isv+1
        ds_tem2 = ds_tem2.where(~cond,drop=True)  #data with repeat obs removed
    ds_mn2.to_netcdf(fout)

In [ ]:
#ds= xr.open_dataset('F:/data/sat_data/smap/SSS/L2/JPL/V4.3/2015/136\SMAP_L2B_SSS_01530_20150516T034831_R16010_V4.3.h5')